In [1]:
# Importing libiraries 
import pandas as pd
from sqlalchemy import create_engine
import os 
from urllib.parse import quote_plus

In [23]:
# Reading the datasets
taxi_df = pd.read_csv('green_tripdata_2019-09.csv', low_memory=False)
zone_df = pd.read_csv('taxi+_zone_lookup.csv')

In [24]:
# Display dataframes
taxi_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.5,0.5,0.5,2.36,0.0,NaN,0.3,14.16,1.0,1.0,0.0
1,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
2,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.0,0.5,0.5,0.00,0.0,NaN,0.3,13.30,2.0,1.0,0.0
3,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.5,0.5,0.5,1.50,0.0,NaN,0.3,10.30,1.0,1.0,0.0
4,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.0,0.5,0.5,3.06,0.0,NaN,0.3,18.36,1.0,1.0,0.0


In [25]:
zone_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [26]:
# Getting the column names in each dataframe
taxi_columns = taxi_df.columns

print(taxi_columns)


Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')


In [27]:
zone_columns = zone_df.columns

print(zone_columns)

Index(['LocationID', 'Borough', 'Zone', 'service_zone'], dtype='object')


In [28]:
# Merge the two dataframes 

# Merge taxi_df with zone_df for pickup location
pickup_df = pd.merge(taxi_df, zone_df, left_on='PULocationID', right_on='LocationID', how='left')

# Rename columns from zone_df to indicate pickup location
pickup_df.rename(columns={'Borough': 'PUBorough', 'Zone': 'PUZone', 'service_zone': 'PUservice_zone'}, inplace=True)

# Merge taxi_df with zone_df for dropoff location
final_df = pd.merge(pickup_df, zone_df, left_on='DOLocationID', right_on='LocationID', how='left', suffixes=('_pickup', '_dropoff'))

# Rename columns from zone_df to indicate dropoff location
final_df.rename(columns={'Borough': 'DOBorough', 'Zone': 'DOZone', 'service_zone': 'DOservice_zone'}, inplace=True)

# Now final_df contains all the information from both DataFrames based on pickup and dropoff location IDs

In [32]:
# Show the new dataframe
final_df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,trip_type,congestion_surcharge,LocationID_pickup,PUBorough,PUZone,PUservice_zone,LocationID_dropoff,DOBorough,DOZone,DOservice_zone
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.50,...,1.0,0.0,65,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,189,Brooklyn,Prospect Heights,Boro Zone
1,2.0,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1.0,97,225,5.0,3.20,12.00,...,1.0,0.0,97,Brooklyn,Fort Greene,Boro Zone,225,Brooklyn,Stuyvesant Heights,Boro Zone
2,2.0,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1.0,37,61,5.0,2.99,12.00,...,1.0,0.0,37,Brooklyn,Bushwick South,Boro Zone,61,Brooklyn,Crown Heights North,Boro Zone
3,2.0,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1.0,145,112,1.0,1.73,7.50,...,1.0,0.0,145,Queens,Long Island City/Hunters Point,Boro Zone,112,Brooklyn,Greenpoint,Boro Zone
4,2.0,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1.0,112,198,1.0,3.42,14.00,...,1.0,0.0,112,Brooklyn,Greenpoint,Boro Zone,198,Queens,Ridgewood,Boro Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449058,NaN,2019-09-30 23:13:00,2019-09-30 23:41:00,NaN,NaN,66,71,NaN,7.84,43.25,...,NaN,NaN,66,Brooklyn,DUMBO/Vinegar Hill,Boro Zone,71,Brooklyn,East Flatbush/Farragut,Boro Zone
449059,NaN,2019-09-30 23:26:00,2019-09-30 23:46:00,NaN,NaN,55,26,NaN,3.56,25.75,...,NaN,NaN,55,Brooklyn,Coney Island,Boro Zone,26,Brooklyn,Borough Park,Boro Zone
449060,NaN,2019-09-30 23:15:00,2019-09-30 23:43:00,NaN,NaN,139,78,NaN,18.47,50.47,...,NaN,NaN,139,Queens,Laurelton,Boro Zone,78,Bronx,East Tremont,Boro Zone
449061,NaN,2019-09-30 23:19:00,2019-10-01 00:06:00,NaN,NaN,242,188,NaN,20.10,58.22,...,NaN,NaN,242,Bronx,Van Nest/Morris Park,Boro Zone,188,Brooklyn,Prospect-Lefferts Gardens,Boro Zone


In [33]:
# Convert the text to datetime
final_df.lpep_pickup_datetime =pd.to_datetime(final_df.lpep_pickup_datetime)
final_df.lpep_dropoff_datetime =pd.to_datetime(final_df.lpep_dropoff_datetime)

In [34]:
# Connect to postgres

# PostgreSQL connection parameters
username = 'postgres'
password = 'P@ssw0rd'  # Replace with your actual password
hostname = '172.19.0.3'
port = '5432'
database_name = 'taxi'

encoded_password = quote_plus(password)

# Construct the connection URL
connection_url = f'postgresql://{username}:{encoded_password}@{hostname}:{port}/{database_name}'

# Create engine and connect to the database
engine = create_engine(connection_url)
engine.connect()

In [35]:
engine.connect()

In [19]:
# Create Script for the Dataframe
print(pd.io.sql.get_schema(final_df,name='green_taxi_data',con=engine))


CREATE TABLE green_taxi_data (
	"VendorID" BIGINT, 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" BIGINT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	ehail_fee FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type BIGINT, 
	trip_type BIGINT, 
	congestion_surcharge FLOAT(53), 
	"LocationID_pickup" BIGINT, 
	"PUBorough" TEXT, 
	"PUZone" TEXT, 
	"PUservice_zone" TEXT, 
	"LocationID_dropoff" BIGINT, 
	"DOBorough" TEXT, 
	"DOZone" TEXT, 
	"DOservice_zone" TEXT
)




In [37]:
# Create the table in the database
final_df.head(n=0).to_sql(name='green_taxi_data',con=engine,if_exists='replace')

0

In [38]:
# Inserting the data in the database 
final_df.to_sql(name='green_taxi_data',con=engine,if_exists='append')

63